In [1]:
import os
import pickle
import multiprocessing as mp
import math
import pandas as pd

In [2]:

base_paths = [
    '/home/combined_everything_FL/run_data/d_snn_atks_defs_surrs',
    '/home/combined_everything_FL/run_data/d_ipm_increased_scale',
    '/home/combined_everything_FL/run_data/d_ann_atks_defs',
    '/home/combined_everything_FL/run_data/d_decrease_gaussian_std'
]
file_paths = [os.path.join(base_path, f) for base_path in base_paths for f in os.listdir(base_path) if f.startswith('exp_')]
class ExpRecord:
    pass

In [3]:
exps = []
for path in file_paths:
    exp = pickle.load(open(path, 'rb'))
    if exp.attack['type'] == 'IPM' and exp.attack['params']['scale'] == 0.1:
        continue
    if exp.attack['type'] == 'GaussRandom' and ('std' not in exp.attack['params'] or exp.attack['params']['std'] != 1.0):
        continue
    # if exp.attack['type'] == 'GaussRandom':
    #     print(exp.attack)
    exps.append(exp)
print(len(exps))

1032


In [6]:
data = []
for exp in exps:
    if len(getattr(exp, 'test_accs', [])) == 0 or getattr(exp, 'checkpointed_epoch', None) != 2000:
        continue
    # Extract the attributes from each experiment
    row = {
        'model': exp.model,
        'data': exp.dataset,
        'surr': exp.snn_hyperparams['surrogate']['type'][:-4] if exp.model.startswith('snn') else 'None',
        'atk': str(exp.attack['type']),
        'def': exp.aggregator['type'],
        'best_acc': max([i[1] for i in exp.test_accs]),
    }

    if row['atk'] == 'SignFlip': 
        row['atk'] = 'S.F.'
    elif row['atk'] == 'LabelFlip': 
        row['atk'] = 'L.F.'
    elif row['atk'] == 'GaussRandom': 
        row['atk'] = 'G.R.'

    if row['def'] == 'SignGuard': 
        row['def'] = 'S.G.'
    elif row['def'] == 'CenterClipping':
        row['def'] = 'C.C.'
    elif row['def'] == 'NormClipping':
        row['def'] = 'N.C.'

    data.append(row)

# Create the DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

,model,data,surr,atk,def,best_acc
0,snn_vgg9,CIFAR10,Rectangle,MinMax,DnC,0.7202
1,snn_vgg9,CIFAR100,Rectangle,MinMax,RFA,0.0191
2,snn_vgg9,CIFAR10,Gaussian,L.F.,Mean,0.4522
3,snn_vgg9,CIFAR100,Triangle,MinMax,DnC,0.4641
4,snn_vgg9,CIFAR10,Rectangle,MinMax,RFA,0.1146


In [7]:
no_atk = df[df['atk'] == 'None']

In [8]:
no_atk

,model,data,surr,atk,def,best_acc
49,snn_fc,MNIST,Gaussian,None,Mean,0.9712
97,snn_vgg9,CIFAR10,Rectangle,None,Mean,0.7210
121,snn_fc,MNIST,Triangle,None,Mean,0.9704
145,snn_vgg9,CIFAR100,Gaussian,None,Mean,0.3978
167,snn_vgg9,CIFAR100,Rectangle,None,Mean,0.3997
200,snn_vgg9,CIFAR10,Gaussian,None,Mean,0.7135
227,snn_fc,FMNIST,Rectangle,None,Mean,0.8667
239,snn_vgg9,CIFAR10,FastSigmoid,None,Mean,0.7146
248,snn_fc,FMNIST,Triangle,None,Mean,0.8623
279,snn_fc,FMNIST,Gaussian,None,Mean,0.8635


In [9]:
a = no_atk.groupby(['model', 'data', 'atk', 'def'], as_index=False)['best_acc'].mean()
a = a.rename(columns={'best_acc': 'baseline'})
grouped_no_atk = a

In [10]:
grouped_no_atk

,model,data,atk,def,baseline
0,ann_fc,FMNIST,None,Mean,0.84560
1,ann_fc,MNIST,None,Mean,0.96890
2,ann_vgg9,CIFAR10,None,Mean,0.77660
3,ann_vgg9,CIFAR100,None,Mean,0.44630
4,snn_fc,FMNIST,None,Mean,0.86482
5,snn_fc,MNIST,None,Mean,0.97064
6,snn_vgg9,CIFAR10,None,Mean,0.73600
7,snn_vgg9,CIFAR100,None,Mean,0.42030


In [11]:
atk_no_def = df[(df['atk'] != 'None') & (df['def'] == 'Mean')]

In [12]:
grouped_atk_no_def = atk_no_def.groupby(['model', 'data', 'atk', 'def'], as_index=False)['best_acc'].mean()
grouped_atk_no_def = grouped_atk_no_def.rename(columns={'best_acc': 'acc_atked'})

In [13]:
grouped_atk_no_def

,model,data,atk,def,acc_atked
0,ann_fc,FMNIST,Fang,Mean,0.81740
1,ann_fc,FMNIST,G.R.,Mean,0.40870
2,ann_fc,FMNIST,IPM,Mean,0.81390
3,ann_fc,FMNIST,L.F.,Mean,0.78760
4,ann_fc,FMNIST,Mimic,Mean,0.84120
5,ann_fc,FMNIST,MinMax,Mean,0.59870
6,ann_fc,FMNIST,S.F.,Mean,0.73670
7,ann_fc,MNIST,Fang,Mean,0.92380
8,ann_fc,MNIST,G.R.,Mean,0.75380
9,ann_fc,MNIST,IPM,Mean,0.94680


In [14]:
a = pd.merge(grouped_no_atk, grouped_atk_no_def, how='left', on=['model', 'data', 'def'])
a['rel_drop'] = (a['baseline'] - a['acc_atked'])/a['baseline']
a['rel_retention'] = a['acc_atked']/a['baseline']
a['drop'] = a['baseline'] - a['acc_atked']
a = a[['model', 'data', 'atk_y', 'baseline', 'acc_atked', 'rel_drop', 'rel_retention', 'drop']]
a = a.rename(columns={'atk_y': 'atk'})
atk_no_atk_cmp = a
atk_no_atk_cmp

,model,data,atk,baseline,acc_atked,rel_drop,rel_retention,drop
0,ann_fc,FMNIST,Fang,0.84560,0.81740,0.033349,0.966651,0.02820
1,ann_fc,FMNIST,G.R.,0.84560,0.40870,0.516675,0.483325,0.43690
2,ann_fc,FMNIST,IPM,0.84560,0.81390,0.037488,0.962512,0.03170
3,ann_fc,FMNIST,L.F.,0.84560,0.78760,0.068590,0.931410,0.05800
4,ann_fc,FMNIST,Mimic,0.84560,0.84120,0.005203,0.994797,0.00440
5,ann_fc,FMNIST,MinMax,0.84560,0.59870,0.291982,0.708018,0.24690
6,ann_fc,FMNIST,S.F.,0.84560,0.73670,0.128784,0.871216,0.10890
7,ann_fc,MNIST,Fang,0.96890,0.92380,0.046548,0.953452,0.04510
8,ann_fc,MNIST,G.R.,0.96890,0.75380,0.222004,0.777996,0.21510
9,ann_fc,MNIST,IPM,0.96890,0.94680,0.022809,0.977191,0.02210


In [15]:
a = atk_no_atk_cmp[atk_no_atk_cmp['model'].str.startswith('ann')]
b = atk_no_atk_cmp[atk_no_atk_cmp['model'].str.startswith('snn')]
a = a[['model', 'data', 'atk', 'rel_drop', 'rel_retention', 'drop']].rename(columns={'rel_drop': 'rel_drop_ann', 'rel_retention': 'retention_ann', 'drop': 'drop_ann'})
b = b[['model', 'data', 'atk', 'rel_drop', 'rel_retention', 'drop']].rename(columns={'rel_drop': 'rel_drop_snn', 'rel_retention': 'retention_snn', 'drop': 'drop_snn'})
c = pd.merge(a,b, how='left', on=['data', 'atk'])[['atk', 'data', 'drop_ann', 'drop_snn', 'rel_drop_ann', 'rel_drop_snn', 'retention_ann', 'retention_snn']]
c['snn_better'] = (c['drop_ann'] > c['drop_snn'])
c['snn_better(rel)'] = (c['retention_ann'] < c['retention_snn'])
c['snn_effective'] = c['drop_ann'] - c['drop_snn']
c.sort_values(['atk', 'data'])
cmp_ann_snn_atk = c

## ANN vs SNN robustness against attacks

In [16]:
cmp_ann_snn_atk.pivot(
    index='data',
    columns='atk', 
    values='snn_effective'
)[['L.F.', 'G.R.', 'S.F.', 'Mimic', 'IPM', 'Fang', 'MinMax']]


atk,L.F.,G.R.,S.F.,Mimic,IPM,Fang,MinMax
data,,,,,,,
CIFAR10,-0.04496,0.06656,0.03160,0.01408,-0.05324,0.05510,0.04650
CIFAR100,-0.03040,0.02748,0.00968,0.02446,-0.02554,0.07068,0.02938
FMNIST,-0.05900,0.04304,-0.00726,-0.01096,0.03200,0.00712,-0.06724
MNIST,-0.03276,-0.30812,-0.01718,-0.00456,0.01552,0.03780,0.61372
